In [ ]:
# Install required Python packages
!pip install --upgrade pip  # Always good to ensure pip is up to date
!pip install onnx onnxruntime git+https://github.com/espressif/esp-ppq.git
!pip install protobuf==3.20.2
!pip install torchvision==0.17.2
!git lfs install
!find imagenet-sample-images/ -mindepth 1 -name ".*" -exec rm -rf {} +
!pip install ultralytics


In [ ]:
"""
NOTE: In case you run into an error, restart the kernel and rerun this cell. Select in the menu bar 'Runtime -> Restart session'
"""

from pathlib import Path
import os
from typing import Final
from PIL import Image
from ppq.api import espdl_quantize_onnx
import torch
from torchvision import transforms
from torch.utils.data import DataLoader, Dataset
from ultralytics import YOLO


In [ ]:
model = YOLO('./models/yolo11n.pt')

torch_model = model.model.eval()

In [ ]:
DEVICE: str = 'cpu'

BATCH_SIZE = 1
IMAGE_SIZE = 640

IMAGENET_PATH: Final[Path] = Path('./sample-images')

class ImageFolderDataset(Dataset):
    def __init__(self, image_dir: Path, transform=None):
        self.image_files = [
            image_dir / f for f in os.listdir(image_dir)
            if f.lower().endswith((".jpg", ".jpeg", ".png"))
        ]
        self.transform = transform

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        image = Image.open(self.image_files[idx]).convert("RGB")
        if self.transform:
            image = self.transform(image)
        return image

# Transformation (resize only — no normalization!)
transform = transforms.Compose([
    transforms.Resize((IMAGE_SIZE, IMAGE_SIZE)),
    transforms.ToTensor(),
])

# Create dataset and dataloader
cal_dataset = ImageFolderDataset(IMAGENET_PATH, transform=transform)
cal_dataloader = DataLoader(
    cal_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=0,
    pin_memory=False,
)



In [ ]:
IMAGE_PATH: Final[Path] = Path('./sample-images')


for img_path in IMAGE_PATH.glob("*.jpg"):
    results = model(img_path)
    results[0].save(filename=f"./preds-sample-images/output_{img_path.name}")

In [ ]:
# You need to execute the following command in this github project http://github.com/espressif/esp-dl in this folder models/coco_detect/models/
# python export_onnx.py --weights ./yolo11n.pt --img 640 --batch 1

In [ ]:
ONNX_YOLO_PATH: Final[Path] = Path('./models/yolo11n.onnx')
ESPDL_YOLO_PATH: Final[Path] = Path('./models/yolo11n.espdl')
TARGET_SOC: Final[str] = 'esp32s3'
NUM_OF_BITS: Final[int] = 8


x = cal_dataset[0]
if isinstance(x, (tuple, list)):
    x = x[0]
x = x.unsqueeze(0)

def collate_fn(batch: torch.Tensor) -> torch.Tensor:
    return batch.to(DEVICE)

# make use of ESP-PPQ to quantize the ONNX computation graph, optimize it for ESP32-S3 SoC, and convert it to .espdl
quantized_model = espdl_quantize_onnx(
    onnx_import_file=ONNX_YOLO_PATH.as_posix(),
    espdl_export_file=ESPDL_YOLO_PATH.as_posix(),
    calib_dataloader=cal_dataloader,
    calib_steps=8,
    input_shape=x.shape,
    inputs=None,
    target=TARGET_SOC,
    num_of_bits=NUM_OF_BITS,
    collate_fn=collate_fn,
    dispatching_override=None,
    device=DEVICE,
    error_report=True,
    skip_export=False,
    export_test_values=True,
    verbose=1,
)